# Learning from BEL

This notebook is about learning from BEL graphs

In [1]:
import getpass
import os
import sys
import time

import numpy as np
import pykeen
from pykeen.pipeline import pipeline
from pykeen.triples.leakage import Sealant
from pykeen.triples import TriplesFactory

In [2]:
print(sys.version)

3.8.3 (default, May 27 2020, 20:54:22) 
[Clang 11.0.3 (clang-1103.0.32.59)]


In [3]:
print(time.asctime())

Fri Jul  3 01:04:22 2020


In [4]:
print(getpass.getuser())

cthoyt


In [5]:
print(pykeen.get_version(with_git_hash=True))

1.0.2-dev-62325ee4


In [6]:
ras_triples_path = 'ras_machine_triples.tsv'
missing_ras_triples = not os.path.exists(ras_triples_path)
missing_ras_triples

False

In [7]:
if missing_ras_triples:
    import pybel
    print(pybel.get_version(with_git_hash=True))
    graph = pybel.from_emmaa('rasmachine')
    graph.summarize()
    triples = pybel.to_triples(graph)
    np.savetxt(ras_triples_path, triples, fmt='%s', delimiter='\t')
else:
    triples = np.loadtxt(ras_triples_path, dtype=str, delimiter='\t')

In [8]:
tf = TriplesFactory(triples=triples)

In [9]:
training, testing = tf.split(random_state=1234)

In [10]:
results = pipeline(
    training_triples_factory=training,
    testing_triples_factory=testing,
    model='RotatE',
    training_kwargs=dict(num_epochs=100),
    random_seed=1235,
    device='cpu',
)

INFO:pykeen.training.training_loop:using stopper: <pykeen.stoppers.stopper.NopStopper object at 0x1473aec10>


INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...


INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


INFO:pykeen.evaluation.evaluator:Evaluation took 79.32s seconds


In [11]:
results.metric_results.to_df()

,Type,Metric,Value
0,avg,adjusted_mean_rank,0.533801
1,worst,mean_rank,1079.968031
2,worst,mean_reciprocal_rank,0.035180
3,worst,hits_at_1,0.011935
4,worst,hits_at_3,0.025575
5,worst,hits_at_5,0.040494
6,worst,hits_at_10,0.061381
7,avg,mean_rank,1079.967391
8,avg,mean_reciprocal_rank,0.035180
9,avg,hits_at_1,0.011935
